## Reduction operation: the sum of the numbers in the range [0, value)

In [1]:
import numpy as np
import sys
import cupy as cp#IMPORTAMOS CUPY
import time

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

value = int(sys.argv[1])

X = np.random.rand(value)

# Para imprimir los pimeros valores del array

# print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)


print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

print("Now, the result using numpy.ndarray.sum():", X.sum())




Time taken by reduction operation using a function: 2.62 ms ± 8.27 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
And the result of the sum of numbers in the range [0, value) is: 24995.051343753086

Time taken by reduction operation using numpy.sum(): 7.88 µs ± 8.1 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.sum(): 24995.051343752984 
 
Time taken by reduction operation using numpy.ndarray.sum(): 7.14 µs ± 0.478 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.ndarray.sum(): 24995.051343752984


#3.3. Python HPC: Paralelismo con GPUs
a) Librería cupy: En la siguiente celda de código del notebook9 vamos a utilizar el paquete cupy para acelerar dicha operación de reducción. Como se ha explicado, la libreria cupy es una librería muy
similar a la librería numpy específicamente diseñada para GPUs. De hecho, la mayoría de funciones que hay en numpy tienen el mismo nombre en cupy. Por tanto, de las 2 formas de hacer la suma de
los elementos del array, por medio de la función reduc_operation y por medio de la función sum de la librería numpy, vamos a usar únicamente la función sum de la librería cupy.
Lo que tienes que hacer es modificar el notebook para crear el array en la GPU (usando las funciones de la librería cupy análogas a las de la librería numpy) y utilizar la función sum para calcular la suma de los elementos del array. Como la GPU ya es paralela, no tienes que paralelizar nada más.

In [7]:
import cupy as cp#IMPORTAMOS CUPY
import time

X = cp.random.rand(value)#USAMOS CUPY.random para generar un array aleatorio a través de la gpu
tiempo_init=time.time()
a=cp.sum(X)
tiempo_fin=time.time()
print(f"El tiempo de cupy.sum midiendo con la funcion time.time() time es {tiempo_fin-tiempo_init} segundos")
tiempo = %timeit -r 2 -o -q cp.sum(X)

print("\nTime taken by reduction operation using cupy.sum and %timeit():", tiempo)

print("Now, the result using cupy.sum():", cp.sum(X),"\n ")



El tiempo de cupy.sum midiendo con time es 0.0001354217529296875 segundos
Time taken by reduction operation using numpy.sum(): 17.6 µs ± 68.4 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.sum(): 25047.679554112565 
 


#Apartado d) 
Crea una nueva celda de texto debajo de la última celda de código para explicar los resultados obtenidos por los paquetes cupy y Numba usando la GPU.

Dado que el aparatado b ha sido cancelado realizaremos una comparación de tiempos a usar numpy.sum vs cupy.sum vs reduc_operation(X). 

A continuacion voy a exponer los resultados obtenidos para 10*7 en usando la partición nikola-gpu dado que borh.gpu generaba multiples warnings sobre la presencia de cupy y no podia ejecutarlo. Pero con el mismo scrip de bash y mismo código en nikola si.



SALIDA:
Ejecutamos el scrip de reduction para value = 10000000
Time taken by reduction operation using a function: 863 ms ± 1.91 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 5001228.999118767

Time taken by reduction operation using numpy.sum(): 3.8 ms ± 4.54 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Now, the result using numpy.sum(): 5001228.999118682 
 
Time taken by reduction operation using numpy.ndarray.sum(): 3.76 ms ± 3.06 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Now, the result using numpy.ndarray.sum(): 5001228.999118682
El tiempo de cupy.sum midiendo con la funcion time.time() time es 0.009362459182739258 segundos

Time taken by reduction operation using cupy.sum and %timeit(): 304 µs ± 3.82 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using cupy.sum(): 5000776.320440389 
 

EXPLICACIÓN DE RESULTADOS:
Para comenzar me gustaría comentar que pese a que contruimos la matriz usando el mismo value para random, hay ligeras diferencias. Para la funcion reduc_operation(X) y np.sum no dado que ambos calculan el resultado a partir de la misma matriz. Buscando información he encontrado que cupy maneja precisión simple de 32bits vs numpy que tiene precision doble de 64bits. Puede que esta forma de precisión sumado a que poseen algorimos internos de generación de números aleatiorios sean las responsables de las diferencias entre las sumas.

Respecto a lo que nos concierne que es la optimización de los tiempos de cómputo:
-La función reduc_operation(X), es una operación secuencial de python que no compilamos previamente por esta razon el tiempo es muy elevado al iterar elemento por elemento en el array.
-Con np.sum, esta función esta muy optimizada cuando se trata de realizar operaciones repetitivas en una matriz como es el caso de la suma. Pasamos de 863 ms 3.8 ms unas 230 veces más rapido y con menos código.
-Finalmente analizamos el tiempo con cupy.random y cupy.sum  usaremos %timeit y time.time(). Cuando comparamos los tiempos con estas dos formas de medir obtenemos 9.36milisegundos vs 304microsegundos. Esto puede puede tener sentido si consideramos que con time medimos también el tiempo de trasferir datos de la cpu a la gpu y la inicialización de CUDA. Por otra parte, con %timeit tenemos el tiempo puro sin las sobrecargas iniciales por los motivos comentados. Ya en materia de comparaciones cupy utilizando la paralelización en la gpu gana a la funcion reduction en unas 2838 veces más rapido y a numpy unas 12,5 veces más rápido. 

Como conclusión: para operaciones sencillas con millones de datos en array, conviene la paralelizacion de estas operaciones mediante el uso de la gpu y la implementación de librerias tan intuitivas como cupy. Dado que si hubieramos usado el for para una matriz cuya duracion hubiera llegado a 1 año con cupy hubieramos tardado tan solo 3 horas(con numpy casi 2 dias), la mejora es impresionante y hace reflexionar sobre la importancia de no solo llegar a un resultado, sino de ser óptimos usando los mismos recursos.
